# Tree of thought prompting

For complex tasks that require exploration or strategic lookahead, traditional or simple prompting techniques fall short. Tree of Thoughts (ToT), a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem solving with language models.

Tree of Thoughts (ToT) is a framework that generalizes over chain-of-thought prompting and encourages exploration over thoughts that serve as intermediate steps for general problem solving with language models. It allows for the generation of multiple reasoning paths, enabling the model to explore different avenues of thought and arrive at more robust solutions.

Tree of Thoughts (ToT) prompting is a framework for LLM inference that allows LLMs to make an informed decision by considering and self-evaluating multiple different reasoning paths that will likely lead to an optimal solution. ToT also empowers the model to backtrack when a path is unlikely to lead to a valid solution. ToT is similar to the best-first search algorithm in Computer Science.

ToT aims to mimic human's problem-solving approach. Research shows that, given a problem, humans search through a combinatorial problem space - a tree where the nodes represent partial solutions and branches represent operators that modify the nodes. They use heuristics to identify the next branch that guides them closer to the solution. The process continues till the problem concludes.

## Example (without ToT)

In [23]:
from ollama import generate, GenerateResponse
from typing import Iterator

prompt = """
Using the numbers 4, 9, 10, and 13 and four basic operators +, -, /,*, generate an expression that evaluates to 175. you can use all the numbers more than once.
"""

stream: Iterator[GenerateResponse] = generate(
    model="gemma3:12b",
    prompt=prompt,
    stream=True
)
for chunk in stream:
    print(chunk.response, end="", flush=True)

Here's a solution using the numbers 4, 9, 10, and 13 and the basic operators (+, -, *, /) to achieve 175:

**(13 * 9) + (10 * 4) - 1 = 175**

Let's break it down:

*   13 * 9 = 117
*   10 * 4 = 40
*   117 + 40 - 1 = 175

the model hallucinates and generates a wrong answer. The model is not able to backtrack and explore other paths. Now, let's see how ToT can help the model to explore different paths and arrive at the correct answer.

## Example (with ToT)

In [20]:
from ollama import generate, GenerateResponse
from typing import Iterator

prompt = """
Imagine three different experts are answering this question.
They will brainstorm the answer with step by step reasoning carefully and taking all facts into consideration
All experts will write down 1 step of their thinking,
then share it with the group.
They will each critique their response, and the all the responses of others
They will check their answer based on laws of maths
Then all experts will go on to the next step and write down this step of their thinking.
They will keep going through steps until they reach their conclusion taking into account the thoughts of the other experts
If at any time they realise that there is a flaw in their logic they will backtrack to where that flaw occurred
If any expert realises they're wrong at any point then they acknowledges this and start another train of thought.

each expert will use a backtracking approach to solve the problem.

Each expert will assign a likelihood of their current assertion being correct. Anytime any expert figures out the proper solution, other experts will acknowledge this and verify that solution.
If the solution is verified by all experts it will be the final answer.
The question is...

Using the numbers 4, 9, 10, and 13 and four basic operators +, -, /,*, generate an expression that evaluates to 175. you can use all the numbers more than once.
"""

stream: Iterator[GenerateResponse] = generate(
    model="gemma3:12b",
    prompt=prompt,
    stream=True
)
for chunk in stream:
    print(chunk.response, end="", flush=True)

Okay, let's do this! I'm excited to see how we tackle this. Let's designate ourselves:

* **Expert A (Algebraic Focus):** Will prioritize algebraic manipulation and equation-solving approaches.
* **Expert B (Order of Operations Focus):** Will meticulously track order of operations and explore different groupings.
* **Expert C (Creative Exploration Focus):** Will try a broader range of strategies, looking for patterns and less obvious combinations.

Here's our first step:

**Expert A:** My initial thought is to try to aim for a large multiplication to get us closer to 175 quickly. I'll start by considering 13 * 10. That equals 130.  We need to add 45 to that. Can we create 45 using the remaining numbers (4, 9, and 10) and the operators? It seems difficult to do directly. I'll assign a likelihood of 40% to this being a successful path, as getting to that number seems hard.

**Expert B:** I agree that multiplication is likely needed. Let’s also think about the importance of order of opera